In [11]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

# Parâmetros do problema
Lx, Ly = 10.0, 10.0       # Dimensões do domínio
T = 2.0                   # Tempo total
u, v = 1.0, 0.5           # Componentes da velocidade
D = 0.1                   # Coeficiente de difusão

# Parâmetros numéricos
nx, ny = 50, 50           # Pontos na malha
nt = 100                  # Passos de tempo
dx, dy = Lx/(nx-1), Ly/(ny-1)
dt = T/nt

# Malha espacial
x = np.linspace(0, Lx, nx)
y = np.linspace(0, Ly, ny)
X, Y = np.meshgrid(x, y)

# Condição inicial (Gaussiana 2D)
x0, y0 = Lx/4, Ly/4       # Centro inicial
sigma = 0.5               # Largura inicial

C_analitico = np.exp(-((X-x0)**2 + (Y-y0)**2)/(4*D*sigma)) / (4*np.pi*D*sigma)
C_numerico = C_analitico.copy()

# Coeficientes numéricos
alpha_x = u * dt / dx
alpha_y = v * dt / dy
beta_x = D * dt / dx**2
beta_y = D * dt / dy**2

# Armazenamento para animação
C_history = [C_numerico.copy()]

# Simulação
for n in range(nt):
    Cn = C_numerico.copy()
    
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            C_numerico[i,j] = (Cn[i,j] 
                              - alpha_x * (Cn[i,j] - Cn[i-1,j]) 
                              - alpha_y * (Cn[i,j] - Cn[i,j-1])
                              + beta_x * (Cn[i+1,j] - 2*Cn[i,j] + Cn[i-1,j])
                              + beta_y * (Cn[i,j+1] - 2*Cn[i,j] + Cn[i,j-1]))
    
    # Condições de contorno (Dirichlet homogêneas)
    C_numerico[0,:] = 0
    C_numerico[-1,:] = 0
    C_numerico[:,0] = 0
    C_numerico[:,-1] = 0
    
    C_history.append(C_numerico.copy())

# Solução analítica no tempo final
t_final = nt * dt
C_analitico_final = (np.exp(-((X-x0-u*t_final)**2 + (Y-y0-v*t_final)**2)/(4*D*(t_final+sigma))) / (4*np.pi*D*(t_final+sigma))

# Plot 3D da solução final

                     
# Solução numérica
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot da superfície numérica
surf = ax.plot_surface(X, Y, C_history[-1], cmap='viridis', rstride=1, cstride=1, 
                      linewidth=0, antialiased=True)

# Configurações do gráfico
ax.set_title(f'Distribuição de Concentração - Solução Numérica (t={t_final:.2f})', pad=15)
ax.set_xlabel('Coordenada X', labelpad=10)
ax.set_ylabel('Coordenada Y', labelpad=10)
ax.set_zlabel('Concentração', labelpad=10)

# Barra de cores
cbar = fig.colorbar(surf, shrink=0.6, aspect=10, pad=0.1)
cbar.set_label('Nível de Concentração', rotation=270, labelpad=15)

# Ajuste de visualização
ax.view_init(elev=30, azim=45)  # Ângulo de visualização
ax.dist = 10  # Distância da câmera

plt.tight_layout()
plt.show()

# Solução analítica
# ax2 = fig.add_subplot(122, projection='3d')
# surf2 = ax2.plot_surface(X, Y, C_analitico_final, cmap='viridis')
# ax2.set_title(f'Solução Analítica (t={t_final:.2f})')
# ax2.set_xlabel('x')
# ax2.set_ylabel('y')
# ax2.set_zlabel('Concentração')
# fig.colorbar(surf2, ax=ax2, shrink=0.5)

plt.tight_layout()
plt.show()

# # Solução numérica
# ax1 = fig.add_subplot(121, projection='3d')
# surf1 = ax1.plot_surface(X, Y, C_history[-1], cmap='viridis')
# ax1.set_title('Solução Numérica (t={:.2f})'.format(t_final))
# ax1.set_xlabel('x')
# ax1.set_ylabel('y')
# ax1.set_zlabel('Concentração')
# fig.colorbar(surf1, ax=ax1, shrink=0.5)

# # Solução analítica
# ax2 = fig.add_subplot(122, projection='3d')
# surf2 = ax2.plot_surface(X, Y, C_analitico_final, cmap='viridis')
# ax2.set_title('Solução Analítica (t={:.2f})'.format(t_final))
# ax2.set_xlabel('x')
# ax2.set_ylabel('y')
# ax2.set_zlabel('Concentração')
# fig.colorbar(surf2, ax=ax2, shrink=0.5)

# plt.tight_layout()
# plt.show()

# # Animação da evolução
# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(111, projection='3d')

# def update(frame):
#     ax.clear()
#     t = frame * dt
#     # Plot numérico
#     surf = ax.plot_surface(X, Y, C_history[frame], cmap='viridis')
#     ax.set_zlim(0, np.max(C_history[0]))
#     ax.set_title(f'Advecção-Difusão 2D (t = {t:.2f})')
#     ax.set_xlabel('x')
#     ax.set_ylabel('y')
#     ax.set_zlabel('Concentração')
#     return surf,

# ani = FuncAnimation(fig, update, frames=len(C_history), interval=50, blit=False)
# plt.show()

# # Plot de contorno comparativo
# plt.figure(figsize=(12, 5))

# plt.subplot(121)
# plt.contourf(X, Y, C_history[-1], levels=20, cmap='viridis')
# plt.colorbar()
# plt.title('Solução Numérica (t={:.2f})'.format(t_final))
# plt.xlabel('x')
# plt.ylabel('y')

# plt.subplot(122)
# plt.contourf(X, Y, C_analitico_final, levels=20, cmap='viridis')
# plt.colorbar()
# plt.title('Solução Analítica (t={:.2f})'.format(t_final))
# plt.xlabel('x')
# plt.ylabel('y')

# plt.tight_layout()
# plt.show()

SyntaxError: invalid syntax (2302688429.py, line 67)